# Лабораторная работа 3
## Фильтр частиц
### Описание системы

В квадратной площадке длиной 100 метров находится мобильный робот. В углах площадки установлены радиомаяки. Робот может измерять расстояние до маяков. Погрешность измерений описывается нормальным распределением с нулевым математически ожиданием и погрешностью $\sigma = 15$. Известно, что начальное положение робота находится в диапазоне координат (0 -100, 0- 100). Начальная ориентация робота составляет $0 \pm 2$ градуса.

Робот функционирует следующим образом:

- Принимает от оператора требуемый угол поворота и дальность перемещения;
- Поворачивается на заданный угол;
- Проезжает заданную дистанцию;
- Измеряет расстояние до маяков;
- Рассчитывает свое положение.

Поворот робота на заданный угол происходит с нормально распределенной погрешностью с нулевым математическим ожиданием и средневадратическим отклонением $\sigma_{\theta} = 2$. Перемещение робота на заданную дистанцию происходит с нормально распределенной погрешностью с нулевым математическим ожиданием и средневадратическим отклонением $\sigma_{l} = 15$.  Требуется построить фильтр частиц для повышения точности локализации робота.

### Задание

- Задать начальное положение робота и средневадратические отклонения;
- Задать не менее 5 команд на перемещение робота;
- Построить траеткторию движения робота с учетом погрешностей;
- Построить фильтр частиц для робота;
- Построить траеткорию движения робота, положения частиц и оценку положения робота после выполнения каждой из команд;
- Выполнить предыдущие пункты для трех различных начальных положений, среднеквадратических отклонений, количества частиц и траекторий движения.

### Решение

#### 0. Импорт библиотек и создание класса для робота

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Класс робота

class Robot:
    def __init__(self, x, y, ang, sigma_move, sigma_turn, sigma_meas) -> None:
        # Истинное положение робота и угол
        self._true_pos = np.array([[x], 
                                   [y]], np.float_)
        self._calc_pos = np.array([[0], 
                                   [0]], np.float_)
        self._ang = ang
        # среднеквадратическое отклонение при перемещении
        self._sigma_move = sigma_move
        # среднеквадратическое отклонение при повороте
        self._sigma_turn = sigma_turn
        
        # Измерение расстояния до маяка
        self._beacon_meas = np.array([[0],  # 1 маяк
                                      [0],  # 2 маяк
                                      [0],  # 3 маяк
                                      [0]]) # 4 маяк
        # среднеквадратическое отклонение при измерении
        self._sigma_meas = sigma_meas
        
        
    def turn(self, ang):
        self._ang += ang + np.deg2rad(np.random.normal(0, self._sigma_turn))
        
        return self._ang
        
        
    def move(self, path_len):
        true_len = np.float_(path_len + np.random.normal(0, self._sigma_move))
        self._true_pos[0] += true_len * np.cos(self._ang)
        self._true_pos[1] += true_len * np.sin(self._ang)
        
        return self._true_pos
    
    
    def make_measurment(self, beacons):
        #  b3(0,y3)*       * b4 (x4, y4)    # Расположегие маяков
        #                                   #
        #                                   #
        #  b1(0,0) *       * b2 (x2, 0)     #
        meas = np.zeros([4,1])
        for i in range(len(meas)):
            meas[i] = np.sqrt((beacons[i][0] - self._true_pos[0])**2 + (beacons[i][1] - self._true_pos[1])**2) + np.random.normal(0, self._sigma_meas)
        self._beacon_meas = meas
        
        return self._beacon_meas
    
    
    def calc_pos(self, beacons):
        # Вычисление положение при помощи трилатерации по двум маякам
        x_calc = (self._beacon_meas[0]**2 - self._beacon_meas[1]**2 + beacons[1][0]**2)/(2*beacons[1][0])
        y_calc = (self._beacon_meas[0]**2 - self._beacon_meas[2]**2 + beacons[2][1]**2)/(2*beacons[2][1])
        self._calc_pos = np.array([[x_calc],
                                   [y_calc]])
        
        return self._calc_pos
        
        
    def get_true_pos(self):
        return self._true_pos[0], self._true_pos[1]
    
    
    def get_calc_pos(self):
        return self._calc_pos[0], self._calc_pos[1]
    
    
    def get_ang(self):
        return self._ang

#### 1. Задать начальное положение робота и среднеквадратические отклонения

In [3]:
# Характеристики робота
x_0 = 2
y_0 = 4
ang_0 = (np.random.rand() - 0.5)*2

sigma_move = 15
sigma_turn = 2
sigma_meas = 15

# Координаты маяков
# Расположегие маяков

    #  b3(0,y3)*       * b4 (x4, y4)    # 
    #                                   #
    #                                   #
    #  b1(0,0) *       * b2 (x2, 0)     #
    
beacons = np.array([[0,  0],
                    [100,0],
                    [0,  100],
                    [100,100]])


bot = Robot(x_0, y_0, ang_0, sigma_move, sigma_turn, sigma_meas)

#### 2. Задать не менее 5 команд на перемещение робота

In [4]:
# Массивы команд перемещения
cmnd = np.array([[10, np.pi/2],
                 [20, np.pi/4],
                 [10, np.pi/2],
                 [10, -np.pi/2],
                 [10, np.pi/2]])


massive_x = []
massive_y = []
t = []
pos_real_x = 0
pos_real_y = 0

massive_x.append(pos_real_x)

for i in range(5):
    bot.turn(cmnd[i][1])
    bot.move(cmnd[i][0])
    pos_real_x, pos_real_y = bot.get_true_pos()
    massive_x.append(pos_real_x)
    massive_y.append(pos_real_y)
    t.append(i)

#### 3. Построить траеткторию движения робота с учетом погрешностей

In [5]:
#plt.plot(massive, t)
print(massive_x)

[0, array([-49.92538779]), array([-49.92538779]), array([-49.92538779]), array([-49.92538779]), array([-49.92538779])]
